# 1 - Imports

In [1]:
import sys
sys.path.append('/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/scripts/')
import os
import argparse
import matplotlib.pyplot as plt
import random
import torch
import numpy as np
import pickle

from train import Trainer
from settings import TRAIN_DEFAULT_SETTINGS

# 2 - Load the class

In [2]:
default_params_dict = TRAIN_DEFAULT_SETTINGS

default_params_dict['train_labels_path'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/train/tiny_check/train_labels.ndx'
default_params_dict['valid_clients'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/valid/voxceleb_2/clients.ndx'
default_params_dict['valid_impostors'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/valid/voxceleb_2/impostors.ndx'
default_params_dict['eval_and_save_best_model_every'] = 10
default_params_dict['early_stopping'] = 25
default_params_dict['update_optimizer_every'] = 20

default_params = argparse.Namespace(**default_params_dict)

trainer = Trainer(default_params)

22-10-26 11:46:59 - train - INFO - Setting device...
22-10-26 11:47:00 - train - INFO - Running on cuda device.
22-10-26 11:47:00 - train - INFO - 8 GPUs available.
22-10-26 11:47:00 - train - INFO - Device setted.
22-10-26 11:47:00 - train - INFO - Setting random seed...
22-10-26 11:47:00 - train - INFO - Random seed setted.
22-10-26 11:47:00 - train - INFO - Setting params...
22-10-26 11:47:00 - train - INFO - params setted.
22-10-26 11:47:00 - train - INFO - Loading data and labels from /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/train/tiny_check/train_labels.ndx
22-10-26 11:47:00 - train - INFO - Data and labels loaded.
22-10-26 11:47:00 - train - INFO - Loading the network...
22-10-26 11:47:04 - train - INFO - Let's use 8 GPUs!
22-10-26 11:47:04 - train - INFO - Network loaded.
22-10-26 11:47:04 - train - INFO - Loading the loss function...
22-10-26 11:47:04 - train - INFO - Loss function loaded.
22-10-26 11:47:04 - train - INFO - Lo

# 3 - Training process

In [ ]:
trainer.main()

# 4 - Training plots

In [ ]:
trainer.debug_info[0]

In [ ]:
steps = [item["step"] for item in trainer.debug_info]
train_loss = [item["train_loss"] for item in trainer.debug_info]
train_eval_metric = [item["train_eval_metric"] for item in trainer.debug_info]
valid_eval_metric = [item["valid_eval_metric"] for item in trainer.debug_info]
best_train_loss = [item["best_train_loss"] for item in trainer.debug_info]
best_model_train_eval_metric = [item["best_model_train_eval_metric"] for item in trainer.debug_info]
best_model_valid_eval_metric = [item["best_model_valid_eval_metric"] for item in trainer.debug_info]
validations_without_improvement = [item["validations_without_improvement"] for item in trainer.debug_info]

In [ ]:
plt.figure(figsize=(18, 10), dpi = 300)

plt.subplot(4, 1, 1)
plt.plot(steps, train_loss, linewidth = 1, label = "train_loss")
# plt.plot(steps, best_train_loss, "--", label = "best_train_loss")
plt.ylim(0, max(train_loss))
plt.ylabel("Loss")
plt.title("Training loss")

plt.subplot(4, 1, 2)
plt.plot(steps, train_eval_metric, label = "train accuracy")
plt.plot(steps, best_model_train_eval_metric, "--", label = "best_model_train_eval_metric")
plt.ylim(0, 1)
plt.ylabel("Accuracy")
plt.title("Training evaluation")

plt.subplot(4, 1, 3)
plt.plot(steps, valid_eval_metric, label = "validation EER")
plt.plot(steps, best_model_valid_eval_metric, "--", label = "best_model_valid_eval_metric")
plt.ylim(0, 50)
plt.ylabel("EER")
plt.title("Validation evaluation")

plt.subplot(4, 1, 4)
plt.plot(steps, validations_without_improvement, label = "validations_without_improvement")
plt.axhline(trainer.params.early_stopping, linestyle = "--", color = "grey")
plt.ylim(0, trainer.params.early_stopping + 1)
plt.ylabel("validations_without_improvement")
plt.title("Early stoping")

plt.xlabel("Step")

plt.suptitle("Training Process", fontweight = 'bold', fontsize = 16)
plt.legend()
plt.show()

In [ ]:
trainer.load_model()

In [ ]:
trainer.checkpoint.keys()

In [ ]:
trainer.checkpoint['epoch']

In [ ]:
trainer.checkpoint['model_results']

In [ ]:
plt.figure(figsize=(18, 6), dpi = 300)

plt.plot(steps, train_loss, label = "train_loss")
plt.plot(steps, best_train_loss, "--", label = "best_train_loss")

plt.ylim(0, max(train_loss))
plt.xlabel("Step")

plt.title("Training Loss")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(18, 6), dpi = 300)

plt.plot(steps, train_eval_metric, label = "train accuracy")
plt.plot(steps, best_model_train_eval_metric, "--", label = "best_model_train_eval_metric")

plt.ylim(0, 1)
plt.xlabel("Step")

plt.title("Evaluation metrics")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(18, 6), dpi = 300)

plt.plot(steps, valid_eval_metric, label = "valid EER")
plt.plot(steps, best_model_valid_eval_metric, "--", label = "best_model_valid_eval_metric")

plt.ylim(0, 50)
plt.xlabel("Step")

plt.title("Evaluation metrics")
plt.legend()
plt.show()

# 5 - Experiments

In [ ]:
for batch_number, (input, label) in enumerate(trainer.training_generator):
    break

In [15]:
with open(trainer.params.train_labels_path, 'r') as data_labels_file:
    train_labels = data_labels_file.readlines()
    
train_label = train_labels[0]

pickle_path = train_label.replace("\n", "").split(" ")[0] + ".pickle"

with open(pickle_path, 'rb') as pickle_file:
    features_dict = pickle.load(pickle_file)
    
features = features_dict["features"]
features_settings = features_dict["settings"]

In [16]:
features_settings

Namespace(audio_paths_file_folder='./feature_extractor/', audio_paths_file_name='voxceleb_2_dev_feature_extractor_paths.lst', log_file_folder='./logs', log_file_name='feature_extractor_voxceleb_2_dev.log', sampling_rate=16000, n_fft_secs=0.023, window='hamming', win_length_secs=0.023, hop_length_secs=0.01, pre_emph_coef=0.97, n_mels=80, overwrite=True, verbose=False, audio_paths_file_path='./feature_extractor/voxceleb_2_dev_feature_extractor_paths.lst')

In [5]:
train_labels[0]

'/home/usuaris/veussd/DATABASES/VoxCeleb/VoxCeleb2/dev/id06338/O_xVOYPnD9I/00017 0 -1\n'